In [7]:
from __future__ import absolute_import, division, print_function
import nest_asyncio

nest_asyncio.apply()

import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import matplotlib.pyplot as plt

from user import User
from average import Average
from graphing import *
from file_related import *
 
from tensorflow import keras
from sklearn.model_selection import train_test_split

import collections
import warnings

from six.moves import range
import six

SEED = 0

# import os
# os.environ['PYTHONHASHSEED']=str(SEED)
# np.random.seed(SEED)
# import random
# random.seed(SEED)
# tf.set_random_seed(SEED)
# could need to force keras to not use parallelism, see documentation

%load_ext autoreload
%autoreload 2
%matplotlib inline

#@test {"skip": true}

# NOTE: If you are running a Jupyter notebook, and installing a locally built
# pip package, you may need to edit the following to point to the '.whl' file
# on your local filesystem.

# NOTE: The high-performance executor components used in this tutorial are not
# yet included in the released pip package; you may need to compile from source.

# NOTE: Jupyter requires a patch to asyncio.


warnings.simplefilter('ignore')

tf.compat.v1.enable_v2_behavior()

# np.random.seed(0)

# NOTE: If the statement below fails, it means that you are
# using an older version of TFF without the high-performance
# executor stack. Call `tff.framework.set_default_executor()`
# instead to use the default reference runtime.
if six.PY3:
    tff.framework.set_default_executor(tff.framework.create_local_executor())

print(tff.federated_computation(lambda: 'It works!')())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
b'It works!'


In [5]:
import numpy as np
from tensorflow_federated.python.simulation import hdf5_client_data
# https://github.com/tensorflow/federated/blob/master/tensorflow_federated/python/simulation/hdf5_client_data_test.py
# https://github.com/tensorflow/federated/blob/v0.11.0/tensorflow_federated/python/simulation/hdf5_client_data.py
# http://docs.h5py.org/en/stable/high/group.html#Group.create_dataset
# https://stackoverflow.com/questions/55434004/create-a-custom-federated-data-set-in-tensorflow-federated
# https://stackoverflow.com/questions/58965488/how-to-create-federated-dataset-from-a-csv-file

file = "dataset.hdf5"

df = read_file("../dataset/allUsers.lcl.csv")
NUM_CLIENTS = create_hdf5(df,file,0)
"""
def printname(name):
    print(name)
with h5py.File(file,"r") as f:
    pass
    f.visit(printname)
"""

'\ndef printname(name):\n    print(name)\nwith h5py.File(file,"r") as f:\n    pass\n    f.visit(printname)\n'

In [4]:
# emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
myclient = hdf5_client_data.HDF5ClientData(file)
train = myclient
example_dataset = train.create_tf_dataset_for_client(
    train.client_ids[0])

example_element = iter(example_dataset).next()

print(example_element['points'].numpy())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
# NUM_CLIENTS = 10
NUM_EPOCHS = 10
BATCH_SIZE = 20
SHUFFLE_BUFFER = 500

def preprocess(dataset):

    def element_fn(element):
        return collections.OrderedDict([
            ('x', tf.reshape(element['points'], [-1])),
            ('y', tf.reshape(element['label'], [1])),
        ])

    return dataset.repeat(NUM_EPOCHS).map(element_fn).shuffle(
      SHUFFLE_BUFFER).batch(BATCH_SIZE)

In [6]:
type(train)

tensorflow_federated.python.simulation.hdf5_client_data.HDF5ClientData

In [1]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(
    lambda x: x.numpy(), iter(preprocessed_example_dataset).next())

# sample_batch

NameError: name 'preprocess' is not defined

In [8]:
def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x))
          for x in client_ids]

sample_clients = train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(train, sample_clients)

len(federated_train_data), federated_train_data[0]

(14,
 <BatchDataset shapes: OrderedDict([(x, (None, 36)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.float32)])>)

In [9]:
def create_compiled_keras_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(
          10, activation=tf.nn.softmax, kernel_initializer='zeros')])

    model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.02),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model
def init_model(init_seed=None):
    """
    initialise and return a model 
    """
    model = keras.Sequential([
        keras.layers.Flatten(),
#         keras.layers.Dense(4096, activation='relu',
#             kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
#         keras.layers.Dense(1024, activation='relu',
#             kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
#         keras.layers.Dense(128, activation='relu',
#             kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
        keras.layers.Dense(32, activation='relu',
            kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
        keras.layers.Dense(6, activation='softmax',
            kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed))
    ])

    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
#         metrics = ["accuracy"]
    )

    return model


def model_fn():
    keras_model = init_model()
    return tff.learning.from_compiled_keras_model(keras_model, sample_batch)

In [10]:
iterative_process = tff.learning.build_federated_averaging_process(model_fn)
print(str(iterative_process.initialize.type_signature))
state = iterative_process.initialize()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


( -> <model=<trainable=<sequential/dense/kernel=float32[36,32],sequential/dense/bias=float32[32],sequential/dense_1/kernel=float32[32,6],sequential/dense_1/bias=float32[6]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<>,model_broadcast_state=<>>@SERVER)


In [11]:
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics))

round  1, metrics=<sparse_categorical_accuracy=0.9383515119552612,loss=0.5496358871459961>
round  2, metrics=<sparse_categorical_accuracy=0.9629532694816589,loss=0.18527714908123016>
round  3, metrics=<sparse_categorical_accuracy=0.9692147970199585,loss=0.13197511434555054>
round  4, metrics=<sparse_categorical_accuracy=0.9709460139274597,loss=0.11831852793693542>
round  5, metrics=<sparse_categorical_accuracy=0.9749155044555664,loss=0.10432572662830353>
round  6, metrics=<sparse_categorical_accuracy=0.9751062989234924,loss=0.10409539192914963>
round  7, metrics=<sparse_categorical_accuracy=0.9757708311080933,loss=0.09923163801431656>
round  8, metrics=<sparse_categorical_accuracy=0.975578784942627,loss=0.09917626529932022>
round  9, metrics=<sparse_categorical_accuracy=0.9780449867248535,loss=0.09046626836061478>
round 10, metrics=<sparse_categorical_accuracy=0.9793152213096619,loss=0.0887361615896225>
